## Produce driver ranking summary

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
silver_catalog    = "f1_silver"
gold_catalog    = "f1_gold"
adw_catalog = "f1_gold_adw"
silver_schema     = "silver"
gold_schema     = "gold"
adw_schema =      "f1_gold"
gold_table_dlt = "f1_drivers_dlt"
gold_table_par = "f1_drivers_par"


In [1]:
# Step 1: Read managed silver tables
drivers_df = spark.table(f"{silver_catalog}.{silver_schema}.f1_drivers_dlt")

In [1]:
# Step 2: Rename Columns
drivers_final_df = drivers_df \
    .withColumnRenamed("ingestion_date", "update_ts")

In [1]:
drivers_final_df.show()

+---------+----------+----+----------+-----------+------------------+--------------------+
|driver_id|driver_ref|code|       dob|nationality|  driver_full_name|           update_ts|
+---------+----------+----+----------+-----------+------------------+--------------------+
|        1|  hamilton| HAM|1985-01-07|    British|    Lewis Hamilton|2025-11-14 10:45:...|
|        2|  heidfeld| HEI|1977-05-10|     German|     Nick Heidfeld|2025-11-14 10:45:...|
|        3|   rosberg| ROS|1985-06-27|     German|      Nico Rosberg|2025-11-14 10:45:...|
|        4|    alonso| ALO|1981-07-29|    Spanish|   Fernando Alonso|2025-11-14 10:45:...|
|        5|kovalainen| KOV|1981-10-19|    Finnish| Heikki Kovalainen|2025-11-14 10:45:...|
|        6|  nakajima| NAK|1985-01-11|   Japanese|   Kazuki Nakajima|2025-11-14 10:45:...|
|        7|  bourdais| BOU|1979-02-28|     French|Sébastien Bourdais|2025-11-14 10:45:...|
|        8| raikkonen| RAI|1979-10-17|    Finnish|    Kimi Räikkönen|2025-11-14 10:45:...|

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        drivers_final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/drivers")
elif target_type == 'table':
    if target_format == 'parquet':
        drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable(f"{gold_catalog}.{gold_schema}.f1_drivers_dlt")
    elif  target_format == 'delta':
        drivers_final_df.write.mode("overwrite").format("delta").saveAsTable(f"{gold_catalog}.{gold_schema}.f1_drivers_dlt")
    elif target_format == 'adw':
        drivers_final_df.write.insertInto(f"{adw_catalog}.{adw_schema}.f1_drivers")